In [ ]:
import cv2
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import sys
sys.path.append(r"D:\codes\python\courtship\utils")
from cv2_utils import *

%load_ext autoreload
%autoreload 2

In [ ]:
# clips_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\out\sleap_out\20220405_0\flyclips_paired"
clips_dir = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\out\sleap_out\20220405_0\flyclips"
rgb_path = r"D:\Fight\rotation\Wei Zhang\projects\courtship song\data\fromZJC\dvSave-2021_12_29_16_10_58.mp4"


In [ ]:
cap = cv2.VideoCapture(rgb_path)
success = True
frames = []
while(True):
    success, frame = cap.read()
    if not success: break
    frames.append(frame)
frames = np.array(frames)
    

In [ ]:
def annotate_frames(frames, clip):
    frames_clip = frames[clip['frame'].values,]

    x_col_names = filter(lambda s: s.startswith('x_'), clip.columns)
    y_col_names = filter(lambda s: s.startswith('y_'), clip.columns)
    xy_pairs = list(zip(x_col_names, y_col_names))
    xy_colors = np.array([[1,0,0], [0,1,0], [0,0,1], [1,1,0], [1,0,1], [0,1,1]])[:len(xy_pairs)]*255
    for i in range(len(clip)):
        img = frames_clip[i,]
        for xy_pair, color in zip(xy_pairs, xy_colors):
            xy = clip.iloc[i][[xy_pair[0], xy_pair[1]]].values
            xy = np.round(xy.astype(float)).astype(int)
            cv2.circle(img, center=xy, radius=3, color=color.tolist(), thickness=-1)
        # plt.imshow(img)
        # break
        frames_clip[i,] = img

    return frames_clip



for root, _, files in os.walk(clips_dir):
    for file in files:
        if file.endswith('.txt'):
            clip = pd.read_csv(os.path.join(root, file), index_col=0)
            frames_clip = annotate_frames(frames, clip)
            save_frames(os.path.join(root, file.split('.')[0] + '.mp4'), frames_clip)
            break